# Demographics Analysis by Patient Source

This notebook loads NHS diagnostic imaging data from multiple CSV files, applies schema unification and standard cleaning,
then summarises **Top-100 Procedures, Referral Types, and Modalities by Age Band** across different patient-source buckets.

**Steps:**
1. Load & validate CSVs (2024 NHS SW ICBs)
2. Apply schema & patient source cleaning
3. Use ONS-style 5-year age bands
4. For each patient-source bucket:
   - Show Top 100 values by age band
   - Add row-wise percentages
   - Include "Other" and "Total" lines
   - Style with green gradient

**Outputs** are styled pivot tables per bucket.


Imports & Raw-Folder Ingestion

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# ----------- 1A. Point to your raw folder OR hard-code the list ----------
raw_dir   = "/Users/rosstaylor/Downloads/Research Project/Code Folder/nhs-diagnostics-dids-eda/nhs-dids-explorer/data/raw/2024 NHS SW ICBs"
csv_files = glob.glob(os.path.join(raw_dir, "*.csv"))

if not csv_files:
    raise FileNotFoundError("csv_files list is empty – check raw_dir or paths")

print(f"Detected {len(csv_files)} files")

# ----------- canonical 23-column schema from the SQL query ---------------
expected_cols = [
    'icb_code','icb_name','lsoa_code','nhs_region',
    'site_code','site_name','provider_code','provider_name',
    'activity_month','financial_year','financial_month','test_date',
    'age','sex','modality','sub_modality','procedure_name',
    'referral_type','patient_source','cancer_flag','subcancer_flag',
    'referring_org_code','referring_org_name'
]

dfs, meta = [], []
for fp in csv_files:
    peek = pd.read_csv(fp, nrows=5)
    if not set(expected_cols).issubset(peek.columns):
        print(f" {os.path.basename(fp)} – no header found, re-loading with names")
        df_tmp = pd.read_csv(fp, header=None, names=expected_cols, low_memory=False)
    else:
        df_tmp = pd.read_csv(fp, low_memory=False)

    df_tmp = df_tmp.dropna(axis=1, how='all')
    df_tmp.columns = df_tmp.columns.str.strip().str.lower()
    df_tmp = df_tmp[[c for c in expected_cols if c in df_tmp.columns]]
    for col in (set(expected_cols) - set(df_tmp.columns)):
        df_tmp[col] = pd.NA
    df_tmp = df_tmp[expected_cols]
    dfs.append(df_tmp)

    meta.append({
        "file": os.path.basename(fp),
        "rows": len(df_tmp),
        "cols": df_tmp.shape[1],
        "MB": round(df_tmp.memory_usage(deep=True).sum()/1e6, 2)
    })

meta_df = pd.DataFrame(meta)
display(meta_df.style.set_caption("Loaded files – rows / cols / size"))

df = pd.concat(dfs, ignore_index=True)
print(f"Combined shape: {df.shape}")  # expect ~4 M × 23

# Basic type coercion
df["age"] = pd.to_numeric(df["age"], errors="coerce")
df["test_date"] = pd.to_datetime(df["test_date"], errors="coerce")
df["activity_month"] = pd.to_datetime(
    df["activity_month"].astype(str), format="%Y%m", errors="coerce"
)


Detected 7 files
 2024_NHS_SW_Somerset_ICB_11X.csv – no header found, re-loading with names
 2024_NHS_SW_Cornwall_ICB_11N.csv – no header found, re-loading with names
 2024_NHS_SW_Gloucestershire_ICB_11M.csv – no header found, re-loading with names
 2024_NHS_SW_Dorset_ICB_11J.csv – no header found, re-loading with names
 2024_NHS_SW_Devon_ICB_15N.csv – no header found, re-loading with names
 2024_NHS_SW_BSW_ICB_92G.csv – no header found, re-loading with names
 2024_NHS_SW_BNSSG_ICB_15C.csv – no header found, re-loading with names


,file,rows,cols,MB
0,2024_NHS_SW_Somerset_ICB_11X.csv,481843,23,695.270000
1,2024_NHS_SW_Cornwall_ICB_11N.csv,512857,23,748.350000
2,2024_NHS_SW_Gloucestershire_ICB_11M.csv,229186,23,335.490000
3,2024_NHS_SW_Dorset_ICB_11J.csv,525091,23,762.790000
4,2024_NHS_SW_Devon_ICB_15N.csv,676563,23,991.610000
5,2024_NHS_SW_BSW_ICB_92G.csv,741719,23,1106.540000
6,2024_NHS_SW_BNSSG_ICB_15C.csv,821993,23,1240.980000


Combined shape: (3989252, 23)


Clean & Bucket Patient Source

In [2]:
df["patient_source"] = (
    df["patient_source"]
      .astype(str).str.strip().str.lower()
      .str.replace(r"\(this health care provider\)", "", regex=True)
      .str.replace(r"\s+-\s+", " – ", regex=True)
      .replace({"nan": np.nan})
      .fillna("unknown")
)

bucket_map = {
    r"accident|emergency|aed|a&e": "Emergency",
    r"gp direct|gp ":             "GP",
    r"inpatient":                 "Inpatient",
    r"outpatient":                "Outpatient",
    r"elective|planned":          "Elective",
}
def ps_bucket(txt: str) -> str:
    for pat, lab in bucket_map.items():
        if re.search(pat, txt):
            return lab
    return "Other/Unknown"

df["ps_bucket"] = df["patient_source"].apply(ps_bucket)
df["ps_bucket"].value_counts(dropna=False)


ps_bucket
Outpatient       1428095
GP                923522
Emergency         832850
Inpatient         695375
Other/Unknown     109410
Name: count, dtype: int64

ONS 5-Year Age Bands

In [4]:
# Define ONS-style bands
bands = pd.DataFrame({
    "lower":[0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85],
    "upper":[4,9,14,19,24,29,34,39,44,49,54,59,64,69,74,79,84,np.inf],
    "label":["0-4","5-9","10-14","15-19","20-24","25-29","30-34","35-39",
             "40-44","45-49","50-54","55-59","60-64","65-69","70-74",
             "75-79","80-84","85+"]
})
bins   = bands["lower"].tolist() + [np.inf]
labels = bands["label"]

# Keep only whole-number ages
df = df[df["age"].notna() & (df["age"] % 1 == 0)]
df["age"] = df["age"].astype(int)
df["age_band"] = pd.cut(df["age"], bins=bins, labels=labels, right=True)


Columns to Summarise

In [5]:
# --- Columns to include in Top-100 summary ---
cols_to_summarise = {
    'procedure_name': 'SNOMED Procedure',
    'referral_type':  'Referral Type',
    'modality':       'Modality'
}


Helper Function for Top 100

In [6]:
def top100_ageband_summary(sub_df: pd.DataFrame,
                           column: str,
                           label: str,
                           age_band_col: str = 'age_band',
                           max_items: int = 100):
    """
    Build & display a styled pivot table of the Top-N (default 100) values in `column`,
    broken down by age bands. Returns the underlying DataFrame.
    """
    grouped = sub_df.groupby([age_band_col, column]).size().reset_index(name='Count')
    top_values = grouped.groupby(column)['Count'].sum().nlargest(max_items).index
    top_df = grouped[grouped[column].isin(top_values)]

    pivot_counts = top_df.pivot_table(index=column,
                                      columns=age_band_col,
                                      values='Count',
                                      aggfunc='sum',
                                      fill_value=0)

    pivot_counts['Total'] = pivot_counts.sum(axis=1)
    grand_total = pivot_counts['Total'].sum()
    pivot_counts['% of Top 100'] = (pivot_counts['Total'] / grand_total * 100).round(1)
    pivot_counts = pivot_counts.sort_values('Total', ascending=False)

    row_percent = pivot_counts.iloc[:, :-2].div(pivot_counts['Total'], axis=0).mul(100).round(1)
    row_percent.index = [f'{idx} %' for idx in pivot_counts.index]

    interleaved = pd.DataFrame()
    for i in range(len(pivot_counts)):
        interleaved = pd.concat([
            interleaved,
            pivot_counts.iloc[[i]],
            row_percent.iloc[[i]]
        ])

    ageband_totals = sub_df.groupby(age_band_col).size()
    top_totals = pivot_counts.drop(columns=['Total', '% of Top 100']).sum()
    not_counted = ageband_totals - top_totals

    not_row = pd.DataFrame([list(not_counted) + [
                            not_counted.sum(),
                            (not_counted.sum() / grand_total * 100).round(1)
                           ]],
                           index=['Other (not in top 100)'],
                           columns=pivot_counts.columns)

    total_row = pd.DataFrame([list(top_totals) + [grand_total, 100.0]],
                             index=['Total (Top 100 only)'],
                             columns=pivot_counts.columns)

    final_table = pd.concat([interleaved, not_row, total_row])

    styled = (final_table.style
                .set_caption(f'{label} – Top {max_items} by Age Band (row %, totals)')
                .format('{:,}')
                .background_gradient(axis=1, cmap='Greens')
                .set_table_styles([{'selector': 'caption',
                                    'props': [('caption-side', 'top'),
                                              ('font-weight', 'bold'),
                                              ('font-size', '16px')]}]))
    display(styled)
    return final_table


 Loop Over Patient Source Buckets

In [7]:
# --- Generate tables for each ps_bucket ---
results = {}
for bucket in sorted(df['ps_bucket'].unique()):
    print('\n' + '='*80)
    print(f'### Patient-source bucket: {bucket}  '
          f'({(df["ps_bucket"]==bucket).sum():,} records)')
    print('='*80)
    sub = df[df['ps_bucket'] == bucket]
    for col, label in cols_to_summarise.items():
        tbl = top100_ageband_summary(sub, col, label)
        results[(bucket, col)] = tbl



### Patient-source bucket: Emergency  (832,842 records)


/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = sub_df.groupby([age_band_col, column]).size().reset_index(name='Count')
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_counts = top_df.pivot_table(index=column,
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

age_band,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+,Total,% of Top 100
Plain chest X-ray (procedure),"1,434.0",692.0,818.0,"2,552.0","2,975.0","3,406.0","4,125.0","4,295.0","4,459.0","4,777.0","6,516.0","7,935.0","8,606.0","9,130.0","10,942.0","14,294.0","13,008.0","17,753.0","117,717.0",16.4
Plain chest X-ray (procedure) %,1.2,0.6,0.7,2.2,2.5,2.9,3.5,3.6,3.8,4.1,5.5,6.7,7.3,7.8,9.3,12.1,11.1,15.1,nan,nan
Plain chest X-ray (procedure) (399208008),"1,947.0",987.0,893.0,"1,927.0","2,220.0","2,716.0","3,230.0","3,397.0","3,529.0","3,444.0","4,665.0","5,622.0","6,058.0","6,146.0","7,334.0","9,534.0","8,589.0","11,835.0","84,073.0",11.7
Plain chest X-ray (procedure) (399208008) %,2.3,1.2,1.1,2.3,2.6,3.2,3.8,4.0,4.2,4.1,5.5,6.7,7.2,7.3,8.7,11.3,10.2,14.1,nan,nan
Computed tomography of entire head (procedure),226.0,206.0,363.0,"1,153.0","1,296.0","1,477.0","1,701.0","1,632.0","1,863.0","1,894.0","2,500.0","3,050.0","3,273.0","3,477.0","4,655.0","6,626.0","6,897.0","10,685.0","52,974.0",7.4
Computed tomography of entire head (procedure) %,0.4,0.4,0.7,2.2,2.4,2.8,3.2,3.1,3.5,3.6,4.7,5.8,6.2,6.6,8.8,12.5,13.0,20.2,nan,nan
Computed tomography of entire head (procedure) (408754009),342.0,245.0,419.0,993.0,"1,205.0","1,256.0","1,442.0","1,604.0","1,534.0","1,530.0","1,836.0","2,315.0","2,271.0","2,450.0","2,971.0","4,493.0","4,646.0","7,134.0","38,686.0",5.4
Computed tomography of entire head (procedure) (408754009) %,0.9,0.6,1.1,2.6,3.1,3.2,3.7,4.1,4.0,4.0,4.7,6.0,5.9,6.3,7.7,11.6,12.0,18.4,nan,nan
X-ray of right ankle (procedure),200.0,825.0,"1,765.0","1,298.0","1,058.0",944.0,898.0,973.0,802.0,659.0,837.0,783.0,663.0,525.0,479.0,471.0,327.0,421.0,"13,928.0",1.9
X-ray of right ankle (procedure) %,1.4,5.9,12.7,9.3,7.6,6.8,6.4,7.0,5.8,4.7,6.0,5.6,4.8,3.8,3.4,3.4,2.3,3.0,nan,nan


/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = sub_df.groupby([age_band_col, column]).size().reset_index(name='Count')
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_counts = top_df.pivot_table(index=column,
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

age_band,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+,Total,% of Top 100
Consultant,"11,969.0","17,610.0","29,576.0","26,831.0","24,791.0","26,652.0","29,566.0","29,001.0","27,577.0","26,456.0","33,430.0","38,616.0","39,159.0","38,253.0","44,966.0","58,835.0","54,326.0","78,659.0","636,273.0",76.6
Consultant %,1.9,2.8,4.6,4.2,3.9,4.2,4.6,4.6,4.3,4.2,5.3,6.1,6.2,6.0,7.1,9.2,8.5,12.4,nan,nan
Not known,"1,833.0","3,982.0","6,498.0","4,190.0","3,485.0","3,459.0","3,613.0","3,458.0","3,478.0","3,362.0","4,115.0","4,721.0","4,275.0","4,233.0","4,847.0","6,085.0","5,558.0","7,895.0","79,087.0",9.5
Not known %,2.3,5.0,8.2,5.3,4.4,4.4,4.6,4.4,4.4,4.3,5.2,6.0,5.4,5.4,6.1,7.7,7.0,10.0,nan,nan
Dentist,778.0,"1,098.0","2,020.0","2,090.0","1,894.0","1,851.0","2,341.0","2,206.0","2,405.0","2,273.0","3,094.0","3,503.0","3,684.0","3,925.0","4,547.0","6,082.0","5,459.0","6,822.0","56,072.0",6.7
Dentist %,1.4,2.0,3.6,3.7,3.4,3.3,4.2,3.9,4.3,4.1,5.5,6.2,6.6,7.0,8.1,10.8,9.7,12.2,nan,nan
Nurse,"1,331.0","4,135.0","6,739.0","3,649.0","3,236.0","3,348.0","3,499.0","3,112.0","2,853.0","2,626.0","2,970.0","3,312.0","2,967.0","2,552.0","2,474.0","2,604.0","1,915.0","1,855.0","55,177.0",6.6
Nurse %,2.4,7.5,12.2,6.6,5.9,6.1,6.3,5.6,5.2,4.8,5.4,6.0,5.4,4.6,4.5,4.7,3.5,3.4,nan,nan
Other health professionals,52.0,237.0,395.0,192.0,191.0,199.0,201.0,187.0,185.0,164.0,219.0,226.0,231.0,217.0,175.0,176.0,122.0,122.0,"3,491.0",0.4
Other health professionals %,1.5,6.8,11.3,5.5,5.5,5.7,5.8,5.4,5.3,4.7,6.3,6.5,6.6,6.2,5.0,5.0,3.5,3.5,nan,nan


...

age_band,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+,Total,% of Top 100
Ultrasound scan for fetal growth (procedure),0.0,0.0,21.0,"1,597.0","5,502.0","11,568.0","13,995.0","6,940.0","1,586.0",71.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"41,301.0",5.0
Ultrasound scan for fetal growth (procedure) %,0.0,0.0,0.1,3.9,13.3,28.0,33.9,16.8,3.8,0.2,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan,nan
"Computed tomography of thorax, abdomen and pelvis with contrast (procedure)",2.0,2.0,1.0,39.0,87.0,144.0,296.0,457.0,767.0,"1,059.0","1,824.0","2,961.0","3,822.0","4,680.0","5,334.0","5,731.0","3,263.0","1,664.0","32,133.0",3.9
"Computed tomography of thorax, abdomen and pelvis with contrast (procedure) %",0.0,0.0,0.0,0.1,0.3,0.4,0.9,1.4,2.4,3.3,5.7,9.2,11.9,14.6,16.6,17.8,10.2,5.2,nan,nan
Plain chest X-ray (procedure),586.0,408.0,359.0,564.0,573.0,839.0,"1,068.0","1,217.0","1,232.0","1,435.0","1,900.0","2,427.0","2,722.0","2,889.0","3,325.0","3,674.0","2,382.0","1,634.0","29,234.0",3.6
Plain chest X-ray (procedure) %,2.0,1.4,1.2,1.9,2.0,2.9,3.7,4.2,4.2,4.9,6.5,8.3,9.3,9.9,11.4,12.6,8.1,5.6,nan,nan
Bilateral mammography (procedure),0.0,0.0,0.0,0.0,5.0,34.0,168.0,"1,079.0","3,901.0","4,268.0","3,574.0","3,422.0","2,807.0","2,544.0","2,417.0","2,054.0","1,118.0",608.0,"27,999.0",3.4
Bilateral mammography (procedure) %,0.0,0.0,0.0,0.0,0.0,0.1,0.6,3.9,13.9,15.2,12.8,12.2,10.0,9.1,8.6,7.3,4.0,2.2,nan,nan
"Computed tomography of thorax, abdomen and pelvis with contrast (procedure) (433761009)",1.0,0.0,1.0,24.0,40.0,104.0,229.0,405.0,614.0,766.0,"1,518.0","2,272.0","2,750.0","3,409.0","4,057.0","4,435.0","2,286.0","1,098.0","24,009.0",2.9
"Computed tomography of thorax, abdomen and pelvis with contrast (procedure) (433761009) %",0.0,0.0,0.0,0.1,0.2,0.4,1.0,1.7,2.6,3.2,6.3,9.5,11.5,14.2,16.9,18.5,9.5,4.6,nan,nan


/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = sub_df.groupby([age_band_col, column]).size().reset_index(name='Count')
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_counts = top_df.pivot_table(index=column,
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

age_band,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+,Total,% of Top 100
Consultant,"11,388.0","15,762.0","25,422.0","26,479.0","32,134.0","47,922.0","61,085.0","58,728.0","60,774.0","64,970.0","88,390.0","108,227.0","113,960.0","113,143.0","115,748.0","115,173.0","65,412.0","37,289.0","1,162,006.0",81.9
Consultant %,1.0,1.4,2.2,2.3,2.8,4.1,5.3,5.1,5.2,5.6,7.6,9.3,9.8,9.7,10.0,9.9,5.6,3.2,nan,nan
Not known,458.0,887.0,"1,701.0","2,631.0","4,702.0","8,563.0","9,444.0","6,182.0","4,465.0","4,335.0","5,591.0","7,239.0","7,764.0","7,242.0","7,424.0","7,283.0","3,725.0","2,106.0","91,742.0",6.5
Not known %,0.5,1.0,1.9,2.9,5.1,9.3,10.3,6.7,4.9,4.7,6.1,7.9,8.5,7.9,8.1,7.9,4.1,2.3,nan,nan
Other health professionals,79.0,77.0,115.0,"1,745.0","5,025.0","11,043.0","14,059.0","6,760.0","1,765.0",851.0,"1,154.0","1,558.0","1,808.0","1,839.0","2,027.0","2,312.0","1,501.0","1,185.0","54,903.0",3.9
Other health professionals %,0.1,0.1,0.2,3.2,9.2,20.1,25.6,12.3,3.2,1.6,2.1,2.8,3.3,3.3,3.7,4.2,2.7,2.2,nan,nan
Dentist,920.0,"4,140.0","6,225.0","3,033.0","2,047.0","3,018.0","3,274.0","2,113.0","1,155.0",763.0,855.0,"1,054.0","1,070.0",986.0,"1,026.0","1,151.0",995.0,743.0,"34,568.0",2.4
Dentist %,2.7,12.0,18.0,8.8,5.9,8.7,9.5,6.1,3.3,2.2,2.5,3.0,3.1,2.9,3.0,3.3,2.9,2.1,nan,nan
Nurse,34.0,54.0,89.0,637.0,"1,741.0","3,502.0","5,049.0","2,995.0","1,382.0","1,291.0","1,451.0","2,111.0","2,539.0","2,799.0","2,989.0","2,633.0","1,409.0",724.0,"33,429.0",2.4
Nurse %,0.1,0.2,0.3,1.9,5.2,10.5,15.1,9.0,4.1,3.9,4.3,6.3,7.6,8.4,8.9,7.9,4.2,2.2,nan,nan


/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = sub_df.groupby([age_band_col, column]).size().reset_index(name='Count')
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:14: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pivot_counts = top_df.pivot_table(index=column,
/var/folders/sy/gz5gl6d91cbfwd2z85r62rbc0000gn/T/ipykernel_16746/1697639511.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or obser

age_band,0-4,5-9,10-14,15-19,20-24,25-29,30-34,35-39,40-44,45-49,50-54,55-59,60-64,65-69,70-74,75-79,80-84,85+,Total,% of Top 100
Diagnostic ultrasonography (procedure),"3,370.0","2,978.0","3,576.0","10,584.0","23,866.0","46,106.0","56,736.0","34,572.0","17,550.0","14,419.0","19,187.0","21,420.0","20,795.0","19,505.0","19,812.0","20,840.0","12,606.0","8,050.0","355,972.0",28.2
Diagnostic ultrasonography (procedure) %,0.9,0.8,1.0,3.0,6.7,13.0,15.9,9.7,4.9,4.1,5.4,6.0,5.8,5.5,5.6,5.9,3.5,2.3,nan,nan
Plain radiography (procedure),"5,887.0","11,319.0","17,556.0","10,804.0","8,265.0","9,890.0","11,554.0","13,961.0","20,209.0","22,656.0","28,234.0","33,230.0","34,192.0","32,712.0","32,210.0","31,914.0","17,936.0","10,881.0","353,410.0",28.0
Plain radiography (procedure) %,1.7,3.2,5.0,3.1,2.3,2.8,3.3,4.0,5.7,6.4,8.0,9.4,9.7,9.3,9.1,9.0,5.1,3.1,nan,nan
Computerized axial tomography (procedure),253.0,395.0,957.0,"1,679.0","2,119.0","3,004.0","4,651.0","6,039.0","8,056.0","10,076.0","16,969.0","26,403.0","32,112.0","34,746.0","38,734.0","37,888.0","23,150.0","12,743.0","259,974.0",20.6
Computerized axial tomography (procedure) %,0.1,0.2,0.4,0.6,0.8,1.2,1.8,2.3,3.1,3.9,6.5,10.2,12.4,13.4,14.9,14.6,8.9,4.9,nan,nan
Magnetic resonance imaging (procedure),993.0,"2,554.0","5,989.0","6,926.0","7,115.0","9,473.0","12,406.0","14,075.0","14,973.0","15,313.0","19,265.0","22,692.0","22,014.0","20,500.0","19,078.0","17,200.0","7,712.0","3,222.0","221,500.0",17.6
Magnetic resonance imaging (procedure) %,0.4,1.2,2.7,3.1,3.2,4.3,5.6,6.4,6.8,6.9,8.7,10.2,9.9,9.3,8.6,7.8,3.5,1.5,nan,nan
Nuclear medicine procedure (procedure),384.0,261.0,214.0,211.0,239.0,346.0,515.0,612.0,816.0,983.0,"1,629.0","2,177.0","2,752.0","3,470.0","3,976.0","4,413.0","2,502.0","1,221.0","26,721.0",2.1
Nuclear medicine procedure (procedure) %,1.4,1.0,0.8,0.8,0.9,1.3,1.9,2.3,3.1,3.7,6.1,8.1,10.3,13.0,14.9,16.5,9.4,4.6,nan,nan
